In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [4]:
#Handle class imbalance
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [5]:
# Track Experiments Using MLFlow

models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'}, # giving the model parameter, it will be displayed in ml flow ui
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3}, # giving the model parameter, it will be displayed in ml flow ui
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'}, #parameter
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'}, #parameter
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params) # also logging parameters of every models to mlflow
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [9]:
# Initialize MLflow
mlflow.set_experiment("Model Register,Load,Production,Version")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/12/08 18:45:06 INFO mlflow.tracking.fluent: Experiment with name 'Model Register,Load,Production,Version' does not exist. Creating a new experiment.
2025/12/08 18:45:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/08 18:45:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/4/runs/2689218b35ad45fbaae8cc98c55bd28b
🧪 View experiment at: http://localhost:5000/#/experiments/4


2025/12/08 18:45:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/4/runs/ad4adba5c7e14580980464f67a968a0f
🧪 View experiment at: http://localhost:5000/#/experiments/4


2025/12/08 18:45:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/4/runs/4ca8fb60bf2949ceb60b26dd9b2f106a
🧪 View experiment at: http://localhost:5000/#/experiments/4
🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/4/runs/c0e65ee4bd7b4427a484efb8f7d5c3db
🧪 View experiment at: http://localhost:5000/#/experiments/4


In [ ]:
#Now i wanna register one model which performs the best among them.
# Im choosing XGBClassifier With SMOTE model to register
# Register the Model

model_name = 'XGB-Smote' #choose any register name for model
run_id = input("Enter Run ID: ")
model_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri, model_name
)


# after every run version of model changes in mlflow.
#c0e65ee4bd7b4427a484efb8f7d5c3db


Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/12/08 19:25:49 WARNING mlflow.tracking._model_registry.fluent: Run with id c0e65ee4bd7b4427a484efb8f7d5c3db has no artifacts at artifact path 'model', registering model based on models:/m-e8075c662e334198b6358ea6eb513fc3 instead
2025/12/08 19:25:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 4
Created version '4' of model 'XGB-Smote'.


In [ ]:
# Load the Development Model to make predictions
# In mlflow ui i added alias name for version 4 of this model as challenger
# so i dont need to specifc version number. its already clear that we have to pick version 4.
model_name = 'XGB-Smote'
model_version = 1
model_uri = f"models:/{model_name}@challenger"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [ ]:
# Transition the Model to Production
dev_model_uri = f'models:/{model_name}@challenger'
prod_model = 'anomaly-detection-model-management-prod'
client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=dev_model_uri, dst_name=prod_model)


'''
dev_model_uri 
This is the source model you want to copy(developed model).
@challenger means you are referring to the model version currently in the Challenger stage.
Think of this as the development or candidate model that you want to promote.


prod_model 
This is the destination model name in MLflow(production model).
his is the production model where you want to copy the challenger version.

copy_model_version
It takes the model from dev_model_uri (your challenger/dev version)
And creates a new version under prod_model (the production model name).

so we basically took the version 4 of developed (challenger) model ,
then transitioned it to version 1 of production (champion) model

Now if we want we can download the champion model containarize it and deploy.
'''


Successfully registered model 'anomaly-detection-model-management-prod'.
Copied version '4' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-model-management-prod'.


'\ndev_model_uri \nThis is the source model you want to copy(developed model).\n@challenger means you are referring to the model version currently in the Challenger stage.\nThink of this as the development or candidate model that you want to promote.\n\n\nprod_model \nThis is the destination model name in MLflow(production model).\nhis is the production model where you want to copy the challenger version.\n\ncopy_model_version\nIt takes the model from dev_model_uri (your challenger/dev version)\nAnd creates a new version under prod_model (the production model name).\n'

In [25]:


# Load the Production Model to make predictions
# In mlflow ui i added alias name for version 4 of this model as challenger
# so i dont need to specifc version number. its already clear that we have to pick version 4.
model_uri = f"models:/{prod_model}@champion"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])